# An Example 2D optimisation
## Inspired by the following notebook:
[this blog post](https://thuijskens.github.io/2016/12/29/bayesian-optimisation/) referring to 
[this notebook](https://github.com/thuijskens/bayesian-optimization/blob/master/ipython-notebooks/svm-optimization.ipynb)

target function and inspiration for plotting from [here](https://github.com/fmfn/BayesianOptimization/issues/18)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
import seaborn as sns # prettify matplotlib

import numpy as np
import sklearn.gaussian_process as gp

In [ ]:
# local modules
import turbo as tb
import turbo.modules as tm
import turbo.plotting as tp
import turbo.gui as tg

In [ ]:
# Make deterministic
np.random.seed(42)

# Target Function

In [ ]:
xmin, xmax = 0, 6
ymin, ymax = 0, 6
x = np.linspace(xmin, xmax, 100)
y = np.linspace(ymin, ymax, 100)

X, Y = np.meshgrid(x, y)

# vectorize needed for accessing noise
#@np.vectorize
def f(x, y):
    ''' from https://github.com/fmfn/BayesianOptimization/issues/18 '''
    a = np.exp(-( (x - 2)**2/0.7 + (y - 4)**2/1.2) + (x - 2)*(y - 4)/1.6 )
    b = np.exp(-( (x - 4)**2/3 + (y - 2)**2/2.) )
    c = np.exp(-( (x - 4)**2/0.5 + (y - 4)**2/0.5) + (x - 4)*(y - 4)/0.5 )
    d = np.sin(3.1415 * x)
    e = np.exp(-( (x - 5.5)**2/0.5 + (y - 5.5)**2/.5) )
    return 2*a + b - c + 0.17 * d + 2*e

Z = f(X, Y)
best_z = np.max(Z) # ~2.06

tp.surface_3D(X, Y, Z)

# Helper Functions

In [ ]:
bounds = [
    ('x', xmin, xmax),
    ('y', ymin, ymax)
]

op = tb.Optimiser(f, 'max', bounds)
op.latent_space = tm.NoLatentSpace()
op.plan = tm.Plan(pre_phase_trials=3)
op.pre_phase_select = tm.random_selector()
op.maximise_acq = tm.random_quasi_newton(num_random=100, grad_restarts=5)
op.async_eval = None
op.surrogate_factory = tm.SciKitGPSurrogate.Factory(gp_params=dict(
    alpha = 1e-3, # larger => more noise. Default = 1e-10
    kernel = 1.0 * gp.kernels.RBF(length_scale_bounds=(1e-2, 5)),
    n_restarts_optimizer = 10,
    #normalize_y = True,
    copy_X_train = True # make a copy of the training data
))
op.acq_func_factory = tm.UCB.Factory(beta=1)

rec = tp.PlottingRecorder()
op.register_listener(rec)

In [ ]:
tg.OptimiserProgressBar(op)
op.run(max_trials=15)

In [ ]:
tp.plot_error_over_time(op, true_best=best_z)

In [ ]:
tp.plot_surrogate_hyper_params_2D(rec)

In [ ]:
tp.plot_trial_1D(rec, param=None, trial_num=None);

In [ ]:
def plot2D(n, step):
    bo.plot_step_2D('x', 'y', n, true_cost=Z)
op.gui.step_log_slider(bo, plot2D, pre_compute=False);

In [ ]:
bo.scatter_plot('x', 'y')

# Try optimising the same function with random search

In [ ]:
ra = op.RandomSearchOptimiser(ranges, maximise_cost=True)
ra.run_sequential(evaluator, max_jobs=1000)

In [ ]:
ra.plot_cost_over_time(true_best=best_z);

In [ ]:
ra.scatter_plot('x', 'y')